<a href="https://colab.research.google.com/github/marrin27/2024_02_29_SIN2324/blob/main/TL%20Sessions%20de%20laboratori/S2%20Perceptr%C3%B3/digits_perceptro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perceptró aplicat a digits


**Lectura del corpus:** $\;$ comprovem també que les matrius de dades i etiquetes tenes les files i columnes que toca

**Partició del corpus:** $\;$ Creem un split de digits amb un $20\%$ de dades per a test i la resta per a entrenament (training), barallant prèviament les dades d'acord amb una llavor donada per a la generació de nombres aleatoris. Ací, com en tot codi que incloga aleatorietat (que requerisca generar nombres aleatoris), convé fixar la dita llavor per a poder reproduir experiments amb exactitud.

In [3]:
import numpy as np; from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits(); X = digits.data.astype(np.float16); X/=X.max() # esto ultimo normaliza los valores para estar entre 0 y 1
y = digits.target.astype(np.uint).reshape(-1, 1); # convierte las etiquetas de clases en enteros sin signo, y reorganiza el array para que sea de 64x1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
print(X_train.shape, X_test.shape)
# print(X.shape, y.shape, "\n", np.hstack([X, y])[:5, :]) imprime las 5 primeras filas, con los valores de sus atributos + su etiqueta de clase

(1437, 64) (360, 64)


Como funciona reshape con un array sencillo


In [ ]:
import numpy as np; A=np.array([[1,2],[3,4]]).reshape(4,1); A

array([[1],
       [2],
       [3],
       [4]])

**Implementació de Perceptró:** $\;$ torna pesos en notació homogènia, $\mathbf{W}\in\mathbb{R}^{(1+D)\times C};\;$ també el nombre d'errors i iteracions executades

In [4]:
def perceptro(X, y, b=0.1, a=1.0, K=200):
    N, D = X.shape; Y = np.unique(y); C = Y.size; W = np.zeros((1+D, C))
    for k in range(1, K+1):
        E = 0
        for n in range(N):
            xn = np.array([1, *X[n, :]])
            cn = np.squeeze(np.where(Y==y[n]))
            gn = W[:,cn].T @ xn; err = False
            for c in np.arange(C):
                if c != cn and W[:,c].T @ xn + b >= gn:
                    W[:, c] = W[:, c] - a*xn; err = True
            if err:
                W[:, cn] = W[:, cn] + a*xn; E = E + 1
        if E == 0:
            break;
    return W, E, k

**Aprenentatge d'un classificador (lineal) amb Perceptró:** $\;$ Perceptró minimitza el nombre d'errors d'entrenament (amb marge)
$$\mathbf{W}^*=\operatorname*{argmin}_{\mathbf{W}=(\boldsymbol{w}_1,\dotsc,\boldsymbol{w}_C)}\sum_n\;\mathbb{I}\biggl(\max_{c\neq y_n}\;\boldsymbol{w}_c^t\boldsymbol{x}_n+b \;>\; \boldsymbol{w}_{y_n}^t\boldsymbol{x}_n\biggr)$$

In [5]:
W, E, k = perceptro(X_train, y_train)
print("Nombre d'iteracions executades: ", k)
print("Nombre d'errors d'entrenament: ", E)
print("Vectors de pesos de les classes (en columnes i amb notació homogènia):\n", W);

Nombre d'iteracions executades:  111
Nombre d'errors d'entrenament:  0
Vectors de pesos de les classes (en columnes i amb notació homogènia):
 [[-1.490000e+02 -1.820000e+02 -1.480000e+02 -1.520000e+02 -1.330000e+02
  -1.580000e+02 -1.500000e+02 -1.490000e+02 -1.580000e+02 -1.810000e+02]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00]
 [-8.750000e-01 -1.750000e+00  1.250000e+00  4.687500e+00 -2.625000e+00
   4.375000e-01 -1.937500e+00  2.062500e+00 -3.812500e+00 -1.050000e+01]
 [-5.643750e+01 -6.143750e+01 -5.650000e+01 -5.900000e+01 -5.650000e+01
  -2.868750e+01 -6.187500e+01 -5.062500e+01 -4.843750e+01 -5.637500e+01]
 [-1.063750e+02 -9.337500e+01 -1.074375e+02 -1.080625e+02 -1.363125e+02
  -1.145000e+02 -1.116875e+02 -1.003750e+02 -1.144375e+02 -8.893750e+01]
 [-1.108750e+02 -1.546250e+02 -1.094375e+02 -8.437500e+01 -1.203750e+02
  -1.091875e+02 -1.175000e+02 -1.068750e+02 -1.116875e+02 -1

**Càlcul de la taxa d'error en test:**

In [6]:
X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])
y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)
err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
print(f"Taxa d'error en test: {err_test:.1%}")

Taxa d'error en test: 4.4%


In [ ]:
np.argmax([[2,3,1],[4,7,1]])

4

**Ajust del marge:** $\;$ experiment per a aprendre un valor de $b$

In [7]:
for b in (.0, .01, .1, 10, 100):
    W, E, k = perceptro(X_train, y_train, b=b, K=1000)
    X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])
    y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)
    err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
    print(f"b: {b} E: {E} k: {k} err_test: {err_test:.1%}")

b: 0.0 E: 0 k: 112 err_test: 4.2%
b: 0.01 E: 0 k: 130 err_test: 3.6%
b: 0.1 E: 0 k: 111 err_test: 4.4%
b: 10 E: 0 k: 187 err_test: 5.0%
b: 100 E: 0 k: 752 err_test: 4.4%


**Interpretació de resultats:** $\;$ les dades d'entrenament no semblen linealment separables; no està clar que un marge major que zero puga millorar resultats, sobretot perquè sols tenim $30$ mostres de test; amb marge nul ja hem vist que s'obté un error (en test) del $16.7\%$